# --- Day 18: RAM Run ---

The algorithm is fast - it's going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you're faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they'll land in your memory space.

Your memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:

```
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
```
Each byte position is given as an X,Y coordinate, where X is the distance from the left edge of your memory space and Y is the distance from the top edge of your memory space.

You and The Historians are currently in the top left corner of the memory space (at 0,0) and need to reach the exit in the bottom right corner (at 70,70 in your memory space, but at 6,6 in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.

As bytes fall into your memory space, they make that coordinate corrupted. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.

In the above example, if you were to draw the memory space after the first 12 bytes have fallen (using . for safe and # for corrupted), it would look like this:
```
...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
```
You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take 22 steps. Here (marked with O) is one such path:
```
OO.#OOO
.O#OO#O
.OOO#OO
...#OO#
..#OO#.
.#.O#..
#.#OOOO
```
Simulate the first kilobyte (1024 bytes) falling onto your memory space. Afterward, what is the minimum number of steps needed to reach the exit?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=18, year=2024)

samp = """5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0"""
print(samp)

5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0


Summarize the task for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠

This is a pathfinding puzzle:

1. You have a grid (70×70 for the real input, 6×6 for the sample)
2. "Bytes" fall onto coordinates, making them impassable (`#`)
3. You start at (0,0) and need to reach the bottom-right corner
4. You can move up/down/left/right
5. **Task**: After the first 1024 bytes have fallen, find the shortest path length to the exit

It's essentially a BFS (breadth-first search) problem on a grid with obstacles.

Here's the plan:
1. Parse the coordinates of falling blocks. Add them into a numpy array.
2. Use networkx and create an undirected graph.
3. Find the shortest path from top left corner to the bottom right one.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Solid plan! A couple of quick notes:

- For step 2, you'll want to add edges between adjacent cells that are *not* blocked (after the first 1024 bytes fall)
- `nx.shortest_path_length()` will give you the answer directly

Ready to start with step 1? Go ahead and parse those coordinates!

In [ ]:
xs = samp.splitlines()
x = xs[0]
x

'5,4'

In [ ]:
tuple(L(x.split(',')).map(int))

(5, 4)

In [ ]:
def parse_line(x):
    return tuple(L(x.split(',')).map(int))
parse_line(x)

(5, 4)

In [ ]:
def parse(inp):
    return L(inp.splitlines()).map(parse_line)
cs = parse(samp)
cs

[(5, 4), (4, 2), (4, 5), (3, 0), (2, 1), (6, 3), (2, 4), (1, 5), (0, 6), (3, 3), (2, 6), (5, 1), (1, 2), (5, 5), (2, 5), (6, 5), (1, 4), (0, 4), (6, 4), (1, 1), (6, 1), (1, 0), (0, 5), (1, 6), (2, 0)]

In [ ]:
import numpy as np

def create_grid(cs, grid_size=71):
    g = np.full((grid_size, grid_size), '.')
    for x, y in cs: g[y, x] = '#'
    return g

g = create_grid(cs[:12], grid_size=7)
print(g)

[['.' '.' '.' '#' '.' '.' '.']
 ['.' '.' '#' '.' '.' '#' '.']
 ['.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '#' '.' '.' '#']
 ['.' '.' '#' '.' '.' '#' '.']
 ['.' '#' '.' '.' '#' '.' '.']
 ['#' '.' '#' '.' '.' '.' '.']]


In [ ]:
ds = [(0, 1), (0, -1), (1, 0), (-1, 0)]

In [ ]:
np.argwhere(g == '.')[:5]

array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 4],
       [0, 5]])

In [ ]:
g.shape

(7, 7)

In [ ]:
import networkx as nx
G = nx.Graph()
for src_y, src_x in np.argwhere(g == '.'):
    for dy, dx in ds:
        tgt_y, tgt_x = src_y + dy, src_x + dx
        if 0 <= tgt_y < g.shape[0] and 0 <= tgt_x < g.shape[1] and g[tgt_y, tgt_x] != '#':
            G.add_edge((src_y, src_x), (tgt_y, tgt_x))
G.edges

EdgeView([((np.int64(0), np.int64(0)), (np.int64(0), np.int64(1))), ((np.int64(0), np.int64(0)), (np.int64(1), np.int64(0))), ((np.int64(0), np.int64(1)), (np.int64(0), np.int64(2))), ((np.int64(0), np.int64(1)), (np.int64(1), np.int64(1))), ((np.int64(1), np.int64(0)), (np.int64(1), np.int64(1))), ((np.int64(1), np.int64(0)), (np.int64(2), np.int64(0))), ((np.int64(1), np.int64(1)), (np.int64(2), np.int64(1))), ((np.int64(0), np.int64(4)), (np.int64(0), np.int64(5))), ((np.int64(0), np.int64(4)), (np.int64(1), np.int64(4))), ((np.int64(0), np.int64(5)), (np.int64(0), np.int64(6))), ((np.int64(1), np.int64(4)), (np.int64(1), np.int64(3))), ((np.int64(0), np.int64(6)), (np.int64(1), np.int64(6))), ((np.int64(1), np.int64(6)), (np.int64(2), np.int64(6))), ((np.int64(2), np.int64(0)), (np.int64(2), np.int64(1))), ((np.int64(2), np.int64(0)), (np.int64(3), np.int64(0))), ((np.int64(2), np.int64(1)), (np.int64(2), np.int64(2))), ((np.int64(2), np.int64(1)), (np.int64(3), np.int64(1))), ((np

In [ ]:
def create_graph(grid):
    G = nx.Graph()
    for src_y, src_x in np.argwhere(grid == '.'):
        for dy, dx in ds:
            tgt_y, tgt_x = src_y + dy, src_x + dx
            if 0 <= tgt_y < grid.shape[0] and 0 <= tgt_x < grid.shape[1] and grid[tgt_y, tgt_x] != '#':
                G.add_edge((src_y, src_x), (tgt_y, tgt_x))
    return G
G = create_graph(g)

In [ ]:
nx.shortest_path_length(G, source=(0, 0), target=(g.shape[0] - 1, g.shape[1] - 1))

22

Combine the functions above into `solve` function.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠

```python
def solve(inp, n_bytes=1024, grid_size=71):
    cs = parse(inp)
    g = create_grid(cs[:n_bytes], grid_size)
    G = create_graph(g)
    return nx.shortest_path_length(G, source=(0, 0), target=(grid_size - 1, grid_size - 1))

# Test with sample
solve(samp, n_bytes=12, grid_size=7)
```

In [ ]:
def solve(inp, n_bytes=1024, grid_size=71):
    cs = parse(inp)
    g = create_grid(cs[:n_bytes], grid_size)
    G = create_graph(g)
    return nx.shortest_path_length(G, source=(0, 0), target=(grid_size - 1, grid_size - 1))

# Test with sample
solve(samp, n_bytes=12, grid_size=7)

22

In [ ]:
solve(inp)

282

# --- Part Two ---
The Historians aren't as used to moving around in this pixelated universe as you are. You're afraid they're not going to be fast enough to make it to the exit before the path is completely blocked.

To determine how fast everyone needs to go, you need to determine the first byte that will cut off the path to the exit.

In the above example, after the byte at 1,1 falls, there is still a path to the exit:
```
O..#OOO
O##OO#O
O#OO#OO
OOO#OO#
###OO##
.##O###
#.#OOOO
```
However, after adding the very next byte (at 6,1), there is no longer a path to the exit:
```
...#...
.##..##
.#..#..
...#..#
###..##
.##.###
#.#....
```
So, in this example, the coordinates of the first byte that prevents the exit from being reachable are 6,1.

Simulate more of the bytes that are about to corrupt your memory space. What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position? (Provide the answer as two integers separated by a comma with no other characters.)

Summarize the part two for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

**Part 2**: Find the first falling byte that makes the exit unreachable.

You need to identify which byte's coordinates (when it falls) completely blocks all paths from (0,0) to the bottom-right corner. The answer should be in `x,y` format.

A simple approach: binary search over the number of bytes, checking if a path exists at each point.

How can I test reachability using nxgraph?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠

Use `nx.has_path(G, source, target)` — returns `True` if a path exists, `False` otherwise.

And how can I remove an edge from graph?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠

`G.remove_edge(u, v)` — removes the edge between nodes `u` and `v`.

Or to remove a node (and all its edges): `G.remove_node(n)`

In [ ]:
def solve(inp, n_bytes=1024, grid_size=71):
    cs = parse(inp)
    g = create_grid(cs[:n_bytes], grid_size)
    G = create_graph(g)
    for c_x, c_y in cs[n_bytes:]:
        if (c_y, c_x) in G:
            G.remove_node((c_y, c_x))
            if not nx.has_path(G, source=(0, 0), target=(grid_size - 1, grid_size - 1)):
                return f"{c_x},{c_y}"
solve(samp, n_bytes=12, grid_size=7)

'6,1'

In [ ]:
solve(inp)

'64,29'

# Reflect

Can I make the solve function more functional?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! A few ideas:

1. **Use `next()` with a generator** instead of the explicit loop and return:
   ```python
   return next(f"{x},{y}" for x,y in cs[n_bytes:] if ...)
   ```

2. **Use `itertools.dropwhile`** to skip bytes while the path still exists, then take the first one that blocks it.

3. **Extract the path-checking logic** into a small helper function for clarity.

Which approach interests you most?

Let's extract the path-checking logic first.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠

Here's a helper function:

```python
def is_blocked(G, grid_size):
    return not nx.has_path(G, (0, 0), (grid_size - 1, grid_size - 1))
```

Then in your solve function, the check becomes `if is_blocked(G, grid_size): return f"{c_x},{c_y}"`

In [ ]:
def is_blocked(G, grid_size):
    return not nx.has_path(G, (0, 0), (grid_size - 1, grid_size - 1))

In [ ]:
def solve(inp, n_bytes=1024, grid_size=71):
    cs = parse(inp)
    g = create_grid(cs[:n_bytes], grid_size)
    G = create_graph(g)
    for c_x, c_y in cs[n_bytes:]:
        if (c_y, c_x) in G:
            G.remove_node((c_y, c_x))
            if is_blocked(G, grid_size):
                return f"{c_x},{c_y}"
solve(samp, n_bytes=12, grid_size=7)

'6,1'

Let's refactor the function and use `dropwhile`

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The tricky part is `dropwhile` needs a predicate that also mutates the graph. You could create a helper:

```python
def remove_and_check(G, grid_size):
    def f(coord):
        x, y = coord
        if (y, x) in G: G.remove_node((y, x))
        return not is_blocked(G, grid_size)
    return f
```

Then use:
```python
from itertools import dropwhile
result = next(dropwhile(remove_and_check(G, grid_size), cs[n_bytes:]))
```

Does mixing mutation with `dropwhile` feel too hacky, or do you want to try it?

It doesn't make the code more readable.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Agreed — mutation inside `dropwhile` defeats the purpose of functional style.

A cleaner approach might be **binary search**: instead of removing nodes one by one, test whether the path is blocked at different byte counts. This is both more functional (no mutation during iteration) and faster (O(log n) path checks instead of O(n)).

Would you like to explore that direction?

How can I make the `solve` function more readable?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

A few simple readability improvements:

1. **Name the loop body**: Extract the "remove and check" logic into a small function
2. **Use walrus operator**: Combine the check and return
3. **Add early return clarity**: Split finding vs formatting the result

Here's one option — separate finding the blocking byte from formatting:

```python
def find_blocking_byte(G, bytes_to_drop, grid_size):
    for x, y in bytes_to_drop:
        if (y, x) in G:
            G.remove_node((y, x))
            if is_blocked(G, grid_size): return (x, y)
```

Then `solve` just calls this and formats the result. What do you think?

In [ ]:
def find_blocking_byte(G, bytes_to_drop, grid_size):
    for x, y in bytes_to_drop:
        if (y, x) in G:
            G.remove_node((y, x))
            if is_blocked(G, grid_size): return (x, y)

In [ ]:
def solve(inp, n_bytes=1024, grid_size=71):
    cs = parse(inp)
    g = create_grid(cs[:n_bytes], grid_size)
    G = create_graph(g)
    x, y = find_blocking_byte(G, cs[n_bytes:], grid_size)
    return f"{x},{y}"
solve(samp, n_bytes=12, grid_size=7)

'6,1'

In [ ]:
solve(inp)

'64,29'